# Exploratory Data Analysis: Energy Consumption For Chicago In 2010

## Life cycle of machine learning project:
1. Understanding the problem
1. Data collection
1. Data checks to perform 
1. Exploratory data analysis
1. Data pre-preocessing
1. Model Training
1. Choose best model

### 1: Understanding the problem
The point of this project is to show how energy consumption is affected by variables such as time of the year, type of building, size of building, number of people, etc.

### 2: Data Collection
Dataset <a href='https://data.cityofchicago.org/Environment-Sustainable-Development/Energy-Usage-2010/8yq3-m6wp/about_data'>source</a>


### 2.1: Import data and required packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer

### Read and print 5 first rows of dataset

In [2]:
df = pd.read_csv('data/chicago_energy_consumption.csv')
df.head()

,COMMUNITY AREA NAME,CENSUS BLOCK,BUILDING TYPE,BUILDING_SUBTYPE,KWH JANUARY 2010,KWH FEBRUARY 2010,KWH MARCH 2010,KWH APRIL 2010,KWH MAY 2010,KWH JUNE 2010,...,TOTAL POPULATION,TOTAL UNITS,AVERAGE STORIES,AVERAGE BUILDING AGE,AVERAGE HOUSESIZE,OCCUPIED UNITS,OCCUPIED UNITS PERCENTAGE,RENTER-OCCUPIED HOUSING UNITS,RENTER-OCCUPIED HOUSING PERCENTAGE,OCCUPIED HOUSING UNITS
0,Archer Heights,1.703157e+14,Residential,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,89.0,24.0,2.0,71.33,3.87,23.0,0.9582,9.0,0.3910,23.0
1,Ashburn,1.703170e+14,Residential,Multi 7+,7334.0,7741.0,4214.0,4284.0,2518.0,4273.0,...,112.0,67.0,2.0,41.00,1.81,62.0,0.9254,50.0,0.8059,62.0
2,Auburn Gresham,1.703171e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,48.0,3.0,86.00,3.00,34.0,0.7082,23.0,0.6759,34.0
3,Austin,1.703125e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,121.0,56.0,2.0,84.00,2.95,41.0,0.7321,32.0,0.7800,41.0
4,Austin,1.703125e+14,Commercial,Multi < 7,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,23.0,2.0,85.00,3.26,19.0,0.8261,11.0,0.5790,19.0


### Shape of dataset

In [3]:
print('Columns: ',df.shape[1])
print('Rows: ',df.shape[0])

Columns:  73
Rows:  67051


### 2.2: Dataset information

### Dataset description
For an in-depth look at the dataset click <a href='data_description.md'> here</a>

### List of all columns

In [4]:
df.columns.values

array(['COMMUNITY AREA NAME', 'CENSUS BLOCK', 'BUILDING TYPE',
       'BUILDING_SUBTYPE', 'KWH JANUARY 2010', 'KWH FEBRUARY 2010',
       'KWH MARCH 2010', 'KWH APRIL 2010', 'KWH MAY 2010',
       'KWH JUNE 2010', 'KWH JULY 2010', 'KWH AUGUST 2010',
       'KWH SEPTEMBER 2010', 'KWH OCTOBER 2010', 'KWH NOVEMBER 2010',
       'KWH DECEMBER 2010', 'TOTAL KWH', 'ELECTRICITY ACCOUNTS',
       'ZERO KWH ACCOUNTS', 'THERM JANUARY 2010', 'THERM FEBRUARY 2010',
       'THERM MARCH 2010', 'TERM APRIL 2010', 'THERM MAY 2010',
       'THERM JUNE 2010', 'THERM JULY 2010', 'THERM AUGUST 2010',
       'THERM SEPTEMBER 2010', 'THERM OCTOBER 2010',
       'THERM NOVEMBER 2010', 'THERM DECEMBER 2010', 'TOTAL THERMS',
       'GAS ACCOUNTS', 'KWH TOTAL SQFT', 'THERMS TOTAL SQFT',
       'KWH MEAN 2010', 'KWH STANDARD DEVIATION 2010', 'KWH MINIMUM 2010',
       'KWH 1ST QUARTILE 2010', 'KWH 2ND QUARTILE 2010',
       'KWH 3RD QUARTILE 2010', 'KWH MAXIMUM 2010', 'KWH SQFT MEAN 2010',
       'KWH SQFT STAND

In [5]:
# the column 'TERM APRIL 2010' is mispelled
df.rename(columns={'TERM APRIL 2010': 'THERM APRIL 2010'}, inplace=True)

### 3: Data Checks to Perform
- Missing values
- Duplicates
- Different categories present in categorical columns
- Dataset statistics 

#### Missing Values
 Finding missing values


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df.isnull().sum())
    print('Total missing values: ', df.isnull().sum().sum())

COMMUNITY AREA NAME                        0
CENSUS BLOCK                              77
BUILDING TYPE                             77
BUILDING_SUBTYPE                          77
KWH JANUARY 2010                         871
KWH FEBRUARY 2010                        871
KWH MARCH 2010                           871
KWH APRIL 2010                           871
KWH MAY 2010                             871
KWH JUNE 2010                            871
KWH JULY 2010                            871
KWH AUGUST 2010                          871
KWH SEPTEMBER 2010                       871
KWH OCTOBER 2010                         871
KWH NOVEMBER 2010                        871
KWH DECEMBER 2010                        871
TOTAL KWH                                871
ELECTRICITY ACCOUNTS                     871
ZERO KWH ACCOUNTS                          0
THERM JANUARY 2010                      2230
THERM FEBRUARY 2010                     4232
THERM MARCH 2010                        1482
THERM APRI

#### Missing Values
Deleting 871 rows where electric data is not available.

In [7]:
df = df.dropna(subset=['TOTAL KWH'])
print('Total missing values: ', df.isnull().sum().sum())

Total missing values:  100479


#### Missing Values
Using imputation to andle missing values. Numerical values will be handled with the average in each column. Categorical features will be handled with the mode of each column. 

In [8]:
# Creating 2 lists for categrical and numerical columns
cat_col = [i for i in df.columns.values if df[i].dtypes=='O']
num_col = [i for i in df.columns.values if df[i].dtypes=='float64' or df[i].dtypes=='int64']
#cat_col, num_col

'CENCUS BLOCK' should be a categorical data type because it is describing an area, similar to how zipcodes work. Will simply convert this to object data type.

'ELECTRICITY ACCOUNTS' and 'GAS ACCOUNTS' data types are returning as categorical because of the value "Less than 4". For simplcity, will replace "Less than 4" with 3 and then convert to float64. Will also add new boolean column to indicate which rows ave less than 4 accounts


In [9]:
df['GAS ACCOUNTS'].replace('Less than 4', 3, inplace=True)
df['ELECTRICITY ACCOUNTS'].replace('Less than 4', 3, inplace=True)

# Correcting to appropriate data types
df = df.astype({'CENSUS BLOCK':'object','ELECTRICITY ACCOUNTS': 'float64', 'GAS ACCOUNTS': 'float64' })
 
df['GAS ACCOUNTS < 4']= np.where(df['GAS ACCOUNTS'] < 4, True, False)
df['ELECTRIC ACCOUNTS < 4']= np.where(df['ELECTRICITY ACCOUNTS'] < 4, True, False)

In [10]:
df['CENSUS BLOCK'].dtypes

dtype('O')

In [11]:
# Correcting lists
cat_col = [i for i in df.columns.values if df[i].dtypes=='O']
num_col = [i for i in df.columns.values if df[i].dtypes=='float64' or df[i].dtypes=='int64']
bool_col = [i for i in df.columns.values if df[i].dtypes=='bool']
#cat_col, num_col, bool_col

In [12]:
for col in num_col:
    imputer = SimpleImputer(strategy='mean')
    df[col] = imputer.fit_transform(df[col].values.reshape(-1,1))
print('Total missing values: ', df.isnull().sum().sum())

Total missing values:  231


In [13]:
for col in cat_col:
    df[col] = df[col].fillna(df[col].mode()[0]).astype('object')

print('Total missing values: ', df.isnull().sum().sum())

Total missing values:  0


In [14]:
df['CENSUS BLOCK'].mode()[0]

170318363002005.0

 ### Checking Duplicates

In [15]:
df.duplicated().sum()

0

There are no duplicates

### Values inside of categorical columns

In [16]:
for col in cat_col:
    print(col, df[col].unique(), sep='\n')

COMMUNITY AREA NAME
['Ashburn' 'Austin' 'Avondale' 'Belmont Cragin' 'Brighton Park' 'Chatham'
 'Chicago Lawn' 'Clearing' 'Dunning' 'East Side' 'Edgewater' 'Englewood'
 'Gage Park' 'Garfield Ridge' 'Grand Boulevard' 'Hermosa' 'Humboldt Park'
 'Hyde Park' 'Lakeview' 'Lincoln Park' 'Lincoln Square' 'Logan Square'
 'Loop' 'Morgan Park' 'Near North Side' 'Near West Side' 'New City'
 'North Center' 'North Lawndale' 'North Park' 'Norwood Park' "O'Hare"
 'Oakland' 'Portage Park' 'Pullman' 'Rogers Park' 'Roseland'
 'South Chicago' 'South Lawndale' 'South Shore' 'Uptown' 'Irving Park'
 'West Englewood' 'West Garfield Park' 'Albany Park' 'Auburn Gresham'
 'Fuller Park' 'West Ridge' 'Bridgeport' 'West Town' 'Calumet Heights'
 'Woodlawn' 'Archer Heights' 'Burnside' 'East Garfield Park'
 'Lower West Side' 'Montclare' 'Greater Grand Crossing' 'Hegewisch'
 'Jefferson Park' 'Armour Square' 'Beverly' 'Washington Park' 'West Lawn'
 'West Pullman' 'Kenwood' 'Avalon Park' 'Douglas' 'Forest Glen'
 'South De

### Statistics

In [17]:
for col in num_col:
    print(col, ': ', df[col].mean())

KWH JANUARY 2010 :  17581.587790873375
KWH FEBRUARY 2010 :  17376.51384103959
KWH MARCH 2010 :  16242.122151707465
KWH APRIL 2010 :  15956.963795708672
KWH MAY 2010 :  19066.22780296162
KWH JUNE 2010 :  23004.85256875189
KWH JULY 2010 :  24828.906980961016
KWH AUGUST 2010 :  22675.264143245695
KWH SEPTEMBER 2010 :  18564.097159262616
KWH OCTOBER 2010 :  17241.260773647628
KWH NOVEMBER 2010 :  21017.981051677245
KWH DECEMBER 2010 :  23962.983303112724
TOTAL KWH :  237518.76136294953
ELECTRICITY ACCOUNTS :  14.184708371109096
ZERO KWH ACCOUNTS :  5.260320338470837
THERM JANUARY 2010 :  3931.0363468807595
THERM FEBRUARY 2010 :  3431.3988974144045
THERM MARCH 2010 :  2823.020356784307
THERM APRIL 2010 :  1487.9233707587148
THERM MAY 2010 :  972.3173701046617
THERM JUNE 2010 :  615.7146245794639
THERM JULY 2010 :  527.390120967742
THERM AUGUST 2010 :  507.91598974916525
THERM SEPTEMBER 2010 :  507.1356244630165
THERM OCTOBER 2010 :  727.2078970196418
THERM NOVEMBER 2010 :  1452.71648480727


In [18]:
print(df.describe())

       KWH JANUARY 2010  KWH FEBRUARY 2010  KWH MARCH 2010  KWH APRIL 2010  \
count      6.618000e+04       6.618000e+04    6.618000e+04    6.618000e+04   
mean       1.758159e+04       1.737651e+04    1.624212e+04    1.595696e+04   
std        3.482508e+05       3.351910e+05    3.164713e+05    3.118232e+05   
min        0.000000e+00       0.000000e+00    0.000000e+00    0.000000e+00   
25%        1.370000e+03       1.613000e+03    1.586000e+03    1.579000e+03   
50%        3.481500e+03       3.814000e+03    3.681500e+03    3.646000e+03   
75%        7.157000e+03       7.410250e+03    7.059000e+03    7.010000e+03   
max        5.298534e+07       4.787976e+07    4.413646e+07    4.222055e+07   

       KWH MAY 2010  KWH JUNE 2010  KWH JULY 2010  KWH AUGUST 2010  \
count  6.618000e+04   6.618000e+04   6.618000e+04     6.618000e+04   
mean   1.906623e+04   2.300485e+04   2.482891e+04     2.267526e+04   
std    3.634429e+05   3.988582e+05   4.135955e+05     3.940989e+05   
min    0.000000e+

### 4: Additional Infromation

In [19]:
# Using weather libray 'meteostat' to get monthly average temperatures for Chicago 2010
from meteostat import Point, Monthly
from datetime import datetime

# assign variable to proper coordinates
chicago = Point(41.8781, -87.6298, 181)

start = datetime(2010, 1, 1)
end = datetime(2010, 12, 31)

# fetch monthly data
weather = Monthly(chicago, start, end)
weather = weather.fetch()['tavg']

# rename index
weather.index = weather.index.strftime('%B')

#weather

#### Convert Therms to CCFs
Currently the dataset measures natural gas consumption by the energy content(THERMS), however, common industry practice is to measure natural gas by volume(CCF). 


In [20]:
# create lambda function: ccf = therm * 100 
ccf_convert = lambda x: x*100

for col in num_col:
    if 'THERM' in col:
        df[col] = df[col].apply(ccf_convert).round(1)
        df.rename(columns={col:col.replace('THERMS','CCF').replace('THERM','CCF')}, inplace=True)